## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSSSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
18
+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|             coeff_1|             coeff_2|            coeff_3|             coeff_4|             coeff_5|             coeff_6|            coeff_7|             coeff_8|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|              res_4|              res_5|             res_6|             res_7|             res_8|          res_mean|    station|total_var|undefs|              vector|  year|
+--------------------+---------

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'coeff_5', 'coeff_6', 'coeff_7', 'coeff_8', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00047661,1957.0,22.371770
1,USC00047661,1972.0,-0.129008
2,USC00045915,1955.0,-0.129025
3,USC00045915,1957.0,52.088195
4,USC00045915,1967.0,-0.266243


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00040693,USC00043652,USC00045915,USC00046305,USC00047661,USC00049418,USW00023258
year,,,,,,,
1922.0,3.132672,NaN,NaN,NaN,NaN,NaN,NaN
1954.0,NaN,-0.129008,NaN,NaN,NaN,NaN,NaN
1955.0,NaN,NaN,-0.129025,NaN,NaN,NaN,NaN
1957.0,NaN,22.371770,52.088195,NaN,22.371770,NaN,NaN
1962.0,NaN,NaN,NaN,0.238146,NaN,NaN,NaN
1967.0,NaN,NaN,-0.266243,NaN,NaN,NaN,NaN
1968.0,NaN,NaN,52.470898,NaN,NaN,-0.129046,NaN
1969.0,NaN,NaN,60.638523,NaN,NaN,NaN,NaN
1972.0,NaN,NaN,20.265281,NaN,-0.129008,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  24.2455239551
RMS removing mean-by-station=  19.4761616856
RMS removing mean-by-year   =  11.0051786059


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 24.2455239551
0 after removing mean by year    = 11.0051786059
0 after removing mean by stations= 7.14992600849
1 after removing mean by year    = 4.83649868507
1 after removing mean by stations= 3.42384362826
2 after removing mean by year    = 2.55222913471
2 after removing mean by stations= 2.01248591366
3 after removing mean by year    = 1.67807554359
3 after removing mean by stations= 1.47033773
4 after removing mean by year    = 1.34024005082
4 after removing mean by stations= 1.25767783731
